In [6]:
# https://github.com/facebookresearch/hydra/blob/main/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

import napari
import dask.array as da
import napari
from squidpy.im import (
    segment,
    ImageContainer
)

from pathlib import Path


# this file acts as a robust starting point for launching hydra runs and multiruns
# can be run from any place

import hydra
import pyrootutils
from omegaconf import DictConfig

from napari_spongepy import utils
log = utils.get_pylogger(__name__)

# project root setup
# searches for root indicators in parent dirs, like ".git", "pyproject.toml", etc.
# sets PROJECT_ROOT environment variable (used in `configs/paths/default.yaml`)
# loads environment variables from ".env" if exists
# adds root dir to the PYTHONPATH (so this file can be run from any place)
# https://github.com/ashleve/pyrootutils
root = pyrootutils.setup_root(os.getcwd(), dotenv=True, pythonpath=True)

In [8]:
abs_config_dir=os.path.abspath(root / "configs")
with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    cfg = compose(config_name="segment", overrides=["+dataset=vizgen"])
print(OmegaConf.to_yaml(cfg))

subset: []
task_name: segment
tags:
- dev
dataset:
  data_dir: ${paths.data_dir}/vizgen/
  dtype: xarray
  image: ${dataset.data_dir}/20272_slide1_A1-1_DAPI.tiff
  coords: null
segmentation:
  method: watershed
paths:
  root_dir: ${oc.env:PROJECT_ROOT}
  data_dir: ${paths.root_dir}/data/
  log_dir: ${paths.root_dir}/logs/
  output_dir: ${hydra:runtime.output_dir}
  work_dir: ${hydra:runtime.cwd}
  masks: null
extras:
  ignore_warnings: false
  enforce_tags: true
  print_config: true



In [10]:
ic = ImageContainer(img=cfg.dataset.data_dir)
ic

ImageContainer[shape=(1000, 1000), layers=['image']]

In [11]:
segment(ic, layer="image", method="watershed", lazy=True)
ic

ImageContainer[shape=(1000, 1000), layers=['image', 'segmented_watershed']]

In [13]:
seg = ic['segmented_watershed']
seg

<xarray.DataArray 'segmented_watershed' (y: 1000, x: 1000, z: 7, c:0: 1)>
dask.array<stack, shape=(1000, 1000, 7, 1), dtype=uint32, chunksize=(1000, 1000, 1, 1), chunktype=numpy.ndarray>
Coordinates:
  * z        (z) <U1 '0' '1' '2' '3' '4' '5' '6'
Dimensions without coordinates: y, x, c:0
Attributes:
    segmentation:  True

In [16]:
img = ic['image'].transpose('c', 'z', 'y', 'x')
img

<xarray.DataArray 'image' (c: 5, z: 7, y: 1000, x: 1000)>
[35000000 values with dtype=uint16]
Coordinates:
  * z        (z) <U1 '0' '1' '2' '3' '4' '5' '6'
Dimensions without coordinates: c, y, x

In [21]:
seg = ic['segmented_watershed'].transpose('c:0', 'z', 'y', 'x')
seg

<xarray.DataArray 'segmented_watershed' (c:0: 1, z: 7, y: 1000, x: 1000)>
dask.array<transpose, shape=(1, 7, 1000, 1000), dtype=uint32, chunksize=(1, 1, 1000, 1000), chunktype=numpy.ndarray>
Coordinates:
  * z        (z) <U1 '0' '1' '2' '3' '4' '5' '6'
Dimensions without coordinates: c:0, y, x
Attributes:
    segmentation:  True

In [22]:
viewer = napari.view_image(img)
# segmentation is only in z:0 c:0
viewer.add_labels(seg)
napari.run()